In [26]:
import pandas as pd
from joblib import dump,load

names = ['Date', 'Price']
names_2 = ['Date', 'Title', 'Body']

# Read dataset for stocks
df = pd.read_csv('stock.csv', names=names) 
#Check dayWeek
df['Date']=pd.to_datetime(df['Date'])
df['D']=df['Date'].dt.dayofweek
df.iloc[:,2:]=df.iloc[:,2:].astype(float)
df.head()

,Date,Price,D
1537,2013-12-31,110.80,1.0
1538,2013-12-30,111.21,0.0
1539,2013-12-27,112.18,4.0
1540,2013-12-26,111.98,3.0
1541,2013-12-24,111.90,1.0


In [27]:
# Read dataset articles
df_2 = pd.read_csv('news.csv',delimiter=',',names=names_2)


df_2['Date']=pd.to_datetime(df_2['Date'], errors='coerce')
df_2=df_2.dropna()
df_2=df_2.groupby(['Date'],as_index=False).agg(lambda col: ' '.join(col))
df_2['Summary']=0
df_2.head()

,Date,Title,Body,Summary
0,2012-04-23,Crude reverses losses stemming from European g...,Crude oil futures were flat to higher in Asia...,0
1,2012-04-24,Crude rises despite sluggish U.S. housing data...,Crude oil futures rose in Asian trading on We...,0
2,2012-04-25,Crude steady on U.S. Fed growth prospects Oil ...,Crude oil futures erased gains in Asian tradi...,0
3,2012-04-26,"Oil rallies on U.S. pending home sales, fallin...",Crude oil futures rallied higher Thursday as ...,0
4,2012-04-27,Crude up in choppy trading on U.S. GDP data Cr...,Crude oil futures rose in U.S. trading on Fri...,0


In [28]:
#count prices
for y in range(1,6):
    df['%_p'+str(y)]=100*(1-df['Price'].shift(y)/df['Price'])
df.head(5)

,Date,Price,D,%_p1,%_p2,%_p3,%_p4,%_p5
1537,2013-12-31,110.80,1.0,NaN,NaN,NaN,NaN,NaN
1538,2013-12-30,111.21,0.0,0.368672,NaN,NaN,NaN,NaN
1539,2013-12-27,112.18,4.0,0.864682,1.230166,NaN,NaN,NaN
1540,2013-12-26,111.98,3.0,-0.178603,0.687623,1.053760,NaN,NaN
1541,2013-12-24,111.90,1.0,-0.071492,-0.250223,0.616622,0.983021,NaN


In [29]:
import numpy as np
df['Max']=df[["%_p1", "%_p2","%_p3","%_p4","%_p5"]].max(axis=1)
df['Min']=df[["%_p1", "%_p2","%_p3","%_p4","%_p5"]].min(axis=1)
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna()

In [30]:
df['y']=0
df.loc[(df['Min'] <=-2), 'y'] = -1
df.loc[(df['Max'] >=1 ), 'y'] = 1
df.loc[(df['Max'] >=1 ) & (df['Min'] <=-2 ), 'y'] = 2
df.head(5)

,Date,Price,D,%_p1,%_p2,%_p3,%_p4,%_p5,Max,Min,y
1542,2013-12-23,111.56,0.0,-0.304769,-0.376479,-0.555755,0.313733,0.681248,0.681248,-0.555755,0
1543,2013-12-20,111.77,4.0,0.187886,-0.116310,-0.187886,-0.366825,0.501029,0.501029,-0.366825,0
1544,2013-12-19,110.29,3.0,-1.341917,-1.151510,-1.459788,-1.532324,-1.713664,-1.151510,-1.713664,0
1545,2013-12-18,109.63,2.0,-0.602025,-1.952020,-1.760467,-2.070601,-2.143574,-0.602025,-2.143574,-1
1546,2013-12-17,108.44,1.0,-1.097381,-1.706013,-3.070823,-2.877167,-3.190705,-1.097381,-3.190705,-1


In [32]:
#create df_3 

df3= df_2.set_index('Date').join(df.set_index('Date'))
df3['Summary']=df3['Title']+df3['Body']
df3 =df3.dropna()
df3 = df3[['Summary','y']]
X=df3.iloc[:,0]

pd.set_option('display.max_colwidth', -1) 
print(df_2.iloc[1])

print(X.iloc[1])
pd.reset_option('display.max_colwidth')


Date       2012-04-24 00:00:00                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Title      Crude rises despite sluggish U.S. housing data Crude oil higher despite weaker than expected U.S. data Crude oil adds to gains after U.S. data supply report, Fed in focus Crude oil trades flat ahead of EU bond auctions, U.S. supply dat

In [33]:
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
from nltk.corpus import stopwords  

In [34]:
#lemmantizer
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [64]:
#Vectorize

from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=3000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(documents).toarray()  
Y = tfidfconverter.inverse_transform(X)
Y[10:13]

[array(['amid', 'asian', 'asset', 'austerity', 'bounce', 'bounced', 'came',
        'concern', 'cut', 'day', 'december', 'drop', 'election', 'europe',
        'european', 'exposure', 'fall', 'fell', 'greece', 'heavy',
        'impact', 'investor', 'level', 'loss', 'low', 'lowest', 'measure',
        'monday', 'morning', 'negative', 'political', 'post', 'pressure',
        'remains', 'revealed', 'risk', 'riskier', 'selling', 'sharp',
        'sharply', 'since', 'sold', 'though', 'trade', 'trading',
        'tuesday', 'wa', 'wake', 'weekend'], dtype='<U13'),
 array(['amid', 'asian', 'asset', 'austerity', 'bottom', 'came', 'concern',
        'continued', 'cut', 'day', 'debt', 'demand', 'drop', 'due',
        'europe', 'european', 'exchange', 'exposure', 'extend', 'fishing',
        'five', 'global', 'greece', 'greek', 'greenback', 'growth', 'held',
        'investor', 'linked', 'loss', 'low', 'lower', 'measure',
        'mercantile', 'month', 'morning', 'moved', 'near', 'new',
        'ov

In [36]:
y=df3.iloc[:,1]
X, y.head(5)

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.05566236,
         0.08588935],
        [0.        , 0.        , 0.        , ..., 0.        , 0.07211682,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.0532459 ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.0710243 ,
         0.        ]]), Date
 2012-04-23    0.0
 2012-04-24    0.0
 2012-04-25    0.0
 2012-04-26    1.0
 2012-04-27    1.0
 Name: y, dtype: float64)

In [37]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)  
X_test,y_test.head(5)

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.11416441],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]), Date
 2013-06-11   -1.0
 2013-09-20    0.0
 2013-09-23    0.0
 2012-10-05   -1.0
 2013-03-22   -1.0
 Name: y, dtype: float64)

In [38]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test) 

In [59]:
 from sklearn.neural_network import MLPClassifier 
#try also - solver='lbfgs' 
mlp = MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=1000,verbose=True) 
mlp.fit(X_train, y_train) 

Iteration 1, loss = 1.43195908
Iteration 2, loss = 0.90211545
Iteration 3, loss = 0.66495900
Iteration 4, loss = 0.35651155
Iteration 5, loss = 0.24719183
Iteration 6, loss = 0.13979819
Iteration 7, loss = 0.06237723
Iteration 8, loss = 0.03590727
Iteration 9, loss = 0.02226685
Iteration 10, loss = 0.01131100
Iteration 11, loss = 0.00492781
Iteration 12, loss = 0.00253002
Iteration 13, loss = 0.00153606
Iteration 14, loss = 0.00110965
Iteration 15, loss = 0.00091293
Iteration 16, loss = 0.00080161
Iteration 17, loss = 0.00072679
Iteration 18, loss = 0.00068513
Iteration 19, loss = 0.00064710
Iteration 20, loss = 0.00062029
Iteration 21, loss = 0.00059958
Iteration 22, loss = 0.00058248
Iteration 23, loss = 0.00056961
Iteration 24, loss = 0.00055947
Iteration 25, loss = 0.00055092
Iteration 26, loss = 0.00054330
Iteration 27, loss = 0.00053736
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 500, 500), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [60]:
y_pred = mlp.predict(X_test)

In [61]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 


[[ 4 11 12  1]
 [ 2  7  6  0]
 [ 4 14 24  0]
 [ 0  0  2  0]]
              precision    recall  f1-score   support

        -1.0       0.40      0.14      0.21        28
         0.0       0.22      0.47      0.30        15
         1.0       0.55      0.57      0.56        42
         2.0       0.00      0.00      0.00         2

   micro avg       0.40      0.40      0.40        87
   macro avg       0.29      0.30      0.27        87
weighted avg       0.43      0.40      0.39        87

0.40229885057471265
